# Levelable Simplicial Complexes on Finite Simple Graphs

This is the main file for generating graphs and checking the levelable condition for my ISCI 4A12 honours thesis project.

### Notes

* This file supersedes `Experimenting with Jupyter and Sage.ipynb`. That file contains some initial stage experimentation with functions and methods that are implemented here. In particular, a large chunk of that file is concerned with looking for positive solutions to linear systems, adopting an algorithm from Dines (1926). 

* In order to check the levelable condition, we take advantage of the built-in linear programming capabilities in SageMath.

* Outstanding question: should we generate graphs using `graphs()` or `nauty_geng()`? 

In [2]:
# Import relevant things
from sage.graphs.independent_sets import IndependentSets
from sage.graphs.graph_input import from_graph6
import numpy as np
import csv

In [32]:
# Some empty arrays to hold results
g6String = []
solutionExists = []
minSol = []

# The number of vertices
n = 9

# A number less than n
# This is used to split up the search
chunk = 8

connected = True

if connected:
    nautArg = str(n) + " -c "
else:
    nautArg = str(n) + " "

if n >= 9:
    # If we're testing 9 or more vertices, split up the search
    # Refer to the documentation to see how this argument works
    ## It's not explained very clearly...
    nautArg = nautArg + str(chunk) + "/" + str(n)

for g in graphs.nauty_geng(nautArg):
    
    # Record the graph
    g6String.append(g.graph6_string())
    
    # Record the maximal independent set
    indSets = IndependentSets(g, maximal = True)
    
    # Grab the number of facets for graph
    t = len(list(indSets))
    
    # Prepare matrices
    A = np.zeros((t-1, n))
    B = np.zeros((t-1, 1))
    
    for j in range(0,t-1):
    
        # Grab facet j
        Fj = list(indSets)[j]

        # Grab facet j + 1
        Fj1 = list(indSets)[j+1]
 

        # Iterate through every vertex of the facet j
        for k in range(0, len(Fj)):
        
            # Add 1 to these spots in the matrix
            A[j, Fj[k]] = A[j, Fj[k]] + 1
       
        # Iterate through every vertex of the facet j + 1
        for k in range(0, len(Fj1)):
        
            # Subtract 1 to these spots in the matrix
            A[j, Fj1[k]] = A[j, Fj1[k]] - 1
        
        # Other side of the equation - set the j-th entry 
        B[j, 0] = len(Fj) - len(Fj1)
    
    # 
    p = MixedIntegerLinearProgram(maximization = False, solver = "GLPK")
    x = p.new_variable()
    
    for l in range(n):
        p.set_min(x[l], 2)
    
    for l in range(t-1):
        # Add a constraint according to the i^th equation
        p.add_constraint(sum(A[l,j]*x[j] for j in range(n)) == B[l])
    
    try:
        # Solve the system
        p.solve()       
    except:
        # If there is no solution, the program throws an error. Record False in solutionExists if this is the case
        solutionExists.append("F")
        minSol.append("NA")
    else:
        # Get the minimum solution
        s = p.get_values([x[r] for r in range(n)])

        # Double check that the solution works 
        if np.array_equal(np.matrix(np.dot(A,s)), transpose(B)):
            # If we do indeed have a solution, then record True in solutionExists
            solutionExists.append("T")

            # Record the minimum solution
            minSol.append(s)
        else:
            solutionExists.append("Error")
            minSol.append("NA")

In [33]:
if connected:
    filename = "results/resultsconnected" + str(n)
else:
    filename = "results/results" + str(n)
    
if n < 9:
    filename = filename + ".csv"
else:
    filename = filename + "_" + str(chunk + 1) + "of" + str(n) + ".csv"


with open(filename, 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    writer.writerow(['graph6 string'] + ['levelable'] + ['minimized solution'])
    
    for row in range(len(g6String)):
        writer.writerow([g6String[row]] + [solutionExists[row]] + [minSol[row]])


The resulting `.csv` file has 3 columns.

* The first column, `graph6 string`, contains the graph6 string of that graph. In order to read this, you can either use [from_graph6()](http://doc.sagemath.org/html/en/reference/graphs/sage/graphs/graph_input.html) or the [rgraph6 package](https://rdrr.io/rforge/rgraph6/man/asAMatrix.html) in R.
* The second column `levelable` reads `T` if the linear program was able to find a solution that passes the levelable condition for the independence complex for that graph (Thm. 6), and `F` if it can't. This column reads `Error` if the linear program found a solution but it doesn't actually solve the system (this is just for peace of mind).
* The third column `minimized solution` contains the solution given by the linear program (if it exists).